<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/Fine_tuning_Wav2Vec2_for_English_ASR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fine-tuning Wav2Vec2 for English ASR with 🤗 Transformers**

Wav2Vec2 是一种用于自动语音识别（ASR）的预训练模型，由 Alexei Baevski、Michael Auli 和 Alex Conneau 于 [2020年9月](https://ai.facebook.com/blog/wav2vec-20-learning-the-structure-of-speech-from-raw-audio/) 发布。

通过一种新颖的对比预训练目标，Wav2Vec2 从超过50,000小时的未标记语音数据中学习到强大的语音表示。类似于 [BERT的掩码语言建模](http://jalammar.github.io/illustrated-bert/)，该模型通过在将特征向量传递给变换器网络之前随机掩码特征向量来学习上下文化的语音表示。

![wav2vec2_structure](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/wav2vec2.png)

首次证明，预训练后仅需极少量的标记语音数据进行微调，就能达到与最先进的ASR系统相竞争的结果。使用仅10分钟的标记数据，Wav2Vec2 在 [LibriSpeech](https://huggingface.co/datasets/librispeech_asr) 的干净测试集上实现了低于5%的词错误率（WER）——参见 [论文](https://arxiv.org/pdf/2006.11477.pdf) 的表9。

在本笔记本中，我们将详细解释如何在任何英语自动语音识别（ASR）数据集上对 Wav2Vec2 的预训练检查点进行微调。请注意，在本笔记本中，我们将不使用语言模型对 Wav2Vec2 进行微调。将 Wav2Vec2 用作端到端 ASR 系统而不使用语言模型要简单得多，并且已证明独立的 Wav2Vec2 声学模型能够取得令人印象深刻的结果。为了演示目的，我们将在仅包含5小时训练数据的较小 [Timit](https://huggingface.co/datasets/timit_asr) 数据集上对“基础”大小的 [预训练检查点](https://huggingface.co/facebook/wav2vec2-base) 进行微调。

Wav2Vec2 使用连接时序分类（CTC）进行微调，这是一种用于训练神经网络解决序列到序列问题的算法，主要应用于自动语音识别和手写识别。

我强烈推荐阅读 Awni Hannun 撰写的非常出色的博客文章 [使用 CTC 进行序列建模（2017）](https://distill.pub/2017/ctc/)。

请使用GPU L4/A100

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Jul 24 07:49:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   39C    P8             11W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

在开始之前，我们需要从主分支安装 `datasets` 和 `transformers`。此外，我们还需要 `librosa` 包来加载音频文件，以及 `jiwer` 包来使用 [词错误率（WER）](https://huggingface.co/metrics/wer) 指标评估我们的微调模型 ${}^1$。

In [ ]:
!pip install datasets==1.18.3 numpy==1.25.2
!pip install jiwer==3.1.0

In [2]:
!pip list | grep -E "datasets|numpy|evaluate|transformers|librosa|torch|jiwer"

datasets                              1.18.3
evaluate                              0.2.0
jiwer                                 3.1.0
librosa                               0.11.0
numpy                                 1.25.2
sentence-transformers                 4.1.0
tensorflow-datasets                   4.9.9
torch                                 2.6.0+cu124
torchao                               0.10.0
torchaudio                            2.6.0+cu124
torchdata                             0.11.0
torchsummary                          1.5.1
torchtune                             0.6.1
torchvision                           0.21.0+cu124
transformers                          4.53.2
vega-datasets                         0.9.0


我们强烈建议在训练过程中直接将你的训练检查点上传到 [🤗 Hub](https://huggingface.co/)。Hugging Face Hub 集成了版本控制功能，可以确保训练过程中不会丢失任何模型检查点。

为此，你需要保存来自 Hugging Face 网站的认证令牌（如果你还没有账户，请在 [这里](https://huggingface.co/join) 注册！）。

In [3]:
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')


In [4]:
from huggingface_hub import login

login(token=HF_TOKEN)

然后，你需要安装 Git-LFS 以上传你的模型检查点：

In [5]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.





---

${}^1$ Timit 通常使用音素错误率（PER）进行评估，但在自动语音识别（ASR）中，最常用的指标是词错误率（WER）。为了使本笔记本尽可能通用，我们决定使用 WER 来评估模型。

## Prepare Data, Tokenizer, Feature Extractor

自动语音识别（ASR）模型将语音转录为文本，这意味着我们需要一个特征提取器来将语音信号处理为模型的输入格式，例如特征向量，以及一个分词器来将模型的输出格式处理为文本。

在 🤗 Transformers 中，Wav2Vec2 模型因此配备了一个分词器，称为 [Wav2Vec2CTCTokenizer](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#wav2vec2ctctokenizer)，以及一个特征提取器，称为 [Wav2Vec2FeatureExtractor](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#wav2vec2featureextractor)。

让我们首先创建负责解码模型预测的分词器。

### Create Wav2Vec2CTCTokenizer

[预训练的 Wav2Vec2 检查点](https://huggingface.co/facebook/wav2vec2-base) 将语音信号映射到一系列上下文表示，如上图所示。微调后的 Wav2Vec2 检查点需要将这些上下文表示序列映射到相应的转录文本，因此需要在transformer模块之上添加一个线性层（以黄色显示）。这个线性层用于将每个上下文表示分类为一个标记类别，类似于 *例如*，在 BERT 预训练后，在其嵌入上添加一个线性层进行进一步分类——参见 [博客文章](https://huggingface.co/blog/warm-starting-encoder-decoder) 的 *“BERT”* 部分。

该线性层的输出大小对应于词汇表中的标记数量，这 **不** 依赖于 Wav2Vec2 的预训练任务，而仅取决于用于微调的标记数据集。因此，在第一步中，我们将查看 Timit 数据集，并根据数据集的转录文本定义一个词汇表。

我们开始加载数据集并查看其结构。

In [6]:
!mkdir -p datasets

In [6]:
from datasets import load_dataset

timit = load_dataset("timit_asr")

  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
timit

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 1680
    })
})

In [8]:
timit.save_to_disk("datasets")

In [9]:
from datasets import load_dataset

timit = load_dataset("timit_asr")

  0%|          | 0/2 [00:00<?, ?it/s]

许多自动语音识别（ASR）数据集仅为每个音频 `'audio'` 和文件 `'file'` 提供目标文本 `'text'`。Timit 实际上为每个音频文件提供了更多信息，例如 `'phonetic_detail'` 等，这也是为什么许多研究人员在处理 Timit 时选择在音素分类上评估他们的模型，而不是语音识别。然而，为了使本笔记本尽可能通用，我们在微调时将仅考虑转录的文本。

In [10]:
timit = timit.remove_columns(["phonetic_detail", "word_detail", "dialect_region", "id", "sentence_type", "speaker_id"])

In [11]:
timit

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'audio', 'text'],
        num_rows: 1680
    })
})

让我们编写一个简短的函数来显示数据集中的一些随机样本，并运行几次以感受转录文本。

In [12]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [13]:
show_random_elements(timit["train"], num_examples=10)

,file,audio,text
0,/root/.cache/huggingface/datasets/downloads/extracted/404950a46da14eac65eb4e2a8317b1372fb3971d980d91d5d5b221275b1fd7e0/data/TRAIN/DR2/MJEB0/SX26.WAV,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/404950a46da14eac65eb4e2a8317b1372fb3971d980d91d5d5b221275b1fd7e0/data/TRAIN/DR2/MJEB0/SX26.WAV', 'array': [0.00015258789, -3.0517578e-05, 0.00012207031, 3.0517578e-05, 9.1552734e-05, -3.0517578e-05, 6.1035156e-05, 3.0517578e-05, 0.0, 6.1035156e-05, -3.0517578e-05, 9.1552734e-05, 6.1035156e-05, 0.0, -3.0517578e-05, 9.1552734e-05, 9.1552734e-05, 9.1552734e-05, 0.00012207031, 9.1552734e-05, 0.0, 0.0, 0.0, -6.1035156e-05, 0.0, 6.1035156e-05, 0.0, 0.0, 3.0517578e-05, 6.1035156e-05, 0.0, -3.0517578e-05, -3.0517578e-05, 3.0517578e-05, 6.1035156e-05, 6.1035156e-05, 3.0517578e-05, -3.0517578e-05, 6.1035156e-05, 3.0517578e-05, 6.1035156e-05, 0.0, 0.0, 6.1035156e-05, 9.1552734e-05, 0.0, 3.0517578e-05, -6.1035156e-05, 0.0, 9.1552734e-05, 9.1552734e-05, 3.0517578e-05, 6.1035156e-05, 3.0517578e-05, 9.1552734e-05, -3.0517578e-05, 9.1552734e-05, 6.1035156e-05, 3.0517578e-05, 9.1552734e-05, 9.1552734e-05, 0.00012207031, 9.1552734e-05, 6.1035156e-05, 9.1552734e-05, 9.1552734e-05, 0.0, -3.0517578e-05, 9.1552734e-05, -3.0517578e-05, -3.0517578e-05, 0.0, 6.1035156e-05, 0.0, 0.0, 9.1552734e-05, 0.0, 3.0517578e-05, 9.1552734e-05, 6.1035156e-05, 0.0, 0.0, 0.0, 9.1552734e-05, 0.0, 3.0517578e-05, 6.1035156e-05, -6.1035156e-05, 0.0, 0.0, 0.00012207031, 0.0, 6.1035156e-05, 3.0517578e-05, 6.1035156e-05, -3.0517578e-05, 0.0, 6.1035156e-05, 3.0517578e-05, 0.00012207031, ...], 'sampling_rate': 16000}",Most young rise early every morning.
1,/root/.cache/huggingface/datasets/downloads/extracted/404950a46da14eac65eb4e2a8317b1372fb3971d980d91d5d5b221275b1fd7e0/data/TRAIN/DR7/MSES0/SI2216.WAV,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/404950a46da14eac65eb4e2a8317b1372fb3971d980d91d5d5b221275b1fd7e0/data/TRAIN/DR7/MSES0/SI2216.WAV', 'array': [0.0, 0.0, 0.0, 3.0517578e-05, 0.00012207031, 0.0, 3.0517578e-05, 3.0517578e-05, 3.0517578e-05, 0.00012207031, 3.0517578e-05, 6.1035156e-05, 0.00012207031, 9.1552734e-05, 0.00012207031, 0.0, 0.00015258789, 0.0, 0.00012207031, 3.0517578e-05, 0.00012207031, 0.0, 0.00012207031, 9.1552734e-05, 3.0517578e-05, 9.1552734e-05, 9.1552734e-05, 9.1552734e-05, 6.1035156e-05, 0.00015258789, 0.0, 3.0517578e-05, 0.00015258789, 6.1035156e-05, 0.00018310547, 3.0517578e-05, 0.00015258789, 0.00012207031, 0.00012207031, 6.1035156e-05, 3.0517578e-05, 0.0, 0.00012207031, 0.0, 9.1552734e-05, 6.1035156e-05, 0.00018310547, 6.1035156e-05, 0.00015258789, 6.1035156e-05, 0.00015258789, 9.1552734e-05, 6.1035156e-05, 3.0517578e-05, 9.1552734e-05, 9.1552734e-05, 0.00015258789, 9.1552734e-05, 3.0517578e-05, 0.00018310547, 6.1035156e-05, 0.00015258789, 6.1035156e-05, 0.00018310547, 6.1035156e-05, 0.00018310547, 3.0517578e-05, 9.1552734e-05, 3.0517578e-05, 0.00015258789, 9.1552734e-05, 0.0, 6.1035156e-05, 0.0, 0.0, 3.0517578e-05, 3.0517578e-05, 3.0517578e-05, 0.0, 0.00015258789, 0.00012207031, 9.1552734e-05, 0.00012207031, 9.1552734e-05, 6.1035156e-05, 9.1552734e-05, 6.1035156e-05, 0.00012207031, 9.1552734e-05, 6.1035156e-05, 9.1552734e-05, 6.1035156e-05, 0.00018310547, 9.1552734e-05, 6.1035156e-05, 0.00018310547, 3.0517578e-05, -3.0517578e-05, 6.1035156e-05, 3.0517578e-05, ...], 'sampling_rate': 16000}",It's never wrong if love is real.
2,/root/.cache/huggingface/datasets/downloads/extracted/404950a46da14eac65eb4e2a8317b1372fb3971d980d91d5d5b221275b1fd7e0/data/TRAIN/DR3/MJRH1/SX64.WAV,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/404950a46da14eac65eb4e2a8317b1372fb3971d980d91d5d5b221275b1fd7e0/data/TRAIN/DR3/MJRH1/SX64.WAV', 'array': [0.0004272461, -3.0517578e-05, 0.00012207031, -3.0517578e-05, 0.0, -3.0517578e-05, -3.0517578e-05, 0.0, -3.0517578e-05, 0.0, -3.0517578e-05, 0.0, -3.0517578e-05, 3.0517578e-05, 0.0, -3.0517578e-05, -9.1552734e-05, 0.0, -9.1552734e-05, -3.0517578e-05, 0.0, -3.051

好的！转录文本看起来非常干净，语言更像是书面文本而不是对话。这可以理解，因为 [Timit](https://huggingface.co/datasets/timit_asr) 是一个朗读语音语料库。

我们可以看到，转录文本中包含一些特殊字符，例如 `,.?!;:`。在没有语言模型的情况下，将语音片段分类为这些特殊字符要困难得多，因为它们并不真正对应于一个特定的声音单元。例如，字母 `"s"` 有一个较为清晰的声音，而特殊字符 `"."` 则没有。此外，为了理解语音信号的含义，通常不需要在转录中包含特殊字符。

因此，我们将文本标准化，只保留小写字母，并在末尾添加一个单词分隔符标记。

In [14]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [15]:
timit = timit.map(remove_special_characters)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [16]:
show_random_elements(timit["train"].remove_columns(["audio", "file"]))

,text
0,a huge tapestry hung in her hallway
1,the sermon emphasized the need for affirmative action
2,don't ask me to carry an oily rag like that
3,flying standby can be practical if you want to save money
4,success for many turnpikes has come hard
5,it's hard to tell an original from a forgery
6,don't ask me to carry an oily rag like that
7,let's take 'em home
8,meanwhile spring had passed well into summer
9,carl lives in a lively home


很好！这样看起来更好了。我们已经从转录文本中移除了大部分特殊字符，并将其标准化为仅小写字母。

在连接时序分类（CTC）中，通常将语音片段分类为字母，因此我们在这里也将这样做。让我们提取训练和测试数据中的所有不同字母，并从这个字母集合中构建我们的词汇表。

我们编写一个映射函数，将所有转录文本连接成一个长转录文本，然后将该字符串转换为一组字符。重要的是，在 `map(...)` 函数中传递参数 `batched=True`，以便映射函数可以一次性访问所有转录文本。

In [17]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [19]:
vocabs = timit.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=timit.column_names["train"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [20]:
vocabs

DatasetDict({
    train: Dataset({
        features: ['vocab', 'all_text'],
        num_rows: 1
    })
    test: Dataset({
        features: ['vocab', 'all_text'],
        num_rows: 1
    })
})

现在，我们创建训练数据集和测试数据集中所有不同字母的并集，并将结果列表转换为一个枚举字典。

In [21]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

In [22]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'q': 0,
 'k': 1,
 'o': 2,
 'y': 3,
 'n': 4,
 'i': 5,
 'a': 6,
 'f': 7,
 'v': 8,
 'u': 9,
 'h': 10,
 's': 11,
 'b': 12,
 'd': 13,
 'c': 14,
 "'": 15,
 't': 16,
 'm': 17,
 ' ': 18,
 'w': 19,
 'z': 20,
 'p': 21,
 'l': 22,
 'r': 23,
 'x': 24,
 'j': 25,
 'e': 26,
 'g': 27}

很棒，我们看到数据集中包含了字母表中的所有字母（这并不令人惊讶），并且我们还提取了特殊字符 `" "` 和 `'`。需要注意的是，我们没有排除这些特殊字符，原因如下：

- 模型需要学习预测单词何时结束，否则模型预测的将始终是一串字符序列，这将使单词之间无法分隔。
- 在英语中，我们需要保留 `'` 字符来区分不同单词，例如 `"it's"` 和 `"its"`，它们有着截然不同的含义。

为了更清楚地表明 `" "` 拥有自己的标记类别，我们将其替换为更显眼的字符 `|`。此外，我们还添加了一个“未知”标记，以便模型能够处理在 Timit 训练集中未遇到的字符。

最后，我们还添加了一个填充标记，对应于 CTC 算法中的“*空白标记*”。“空白标记”是 CTC 算法的核心组成部分。有关更多信息，请查看[此处](https://distill.pub/2017/ctc/)的“对齐”部分。

In [23]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [24]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

30

In [25]:
vocab_dict

{'q': 0,
 'k': 1,
 'o': 2,
 'y': 3,
 'n': 4,
 'i': 5,
 'a': 6,
 'f': 7,
 'v': 8,
 'u': 9,
 'h': 10,
 's': 11,
 'b': 12,
 'd': 13,
 'c': 14,
 "'": 15,
 't': 16,
 'm': 17,
 'w': 19,
 'z': 20,
 'p': 21,
 'l': 22,
 'r': 23,
 'x': 24,
 'j': 25,
 'e': 26,
 'g': 27,
 '|': 18,
 '[UNK]': 28,
 '[PAD]': 29}

好的，现在我们的词汇表已经完整，包含30个标记，这意味着我们将在预训练的 Wav2Vec2 检查点之上添加的线性层将具有30个输出维度。

好的，我们现在将词汇表保存为 JSON 文件。

In [26]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

最后一步，我们使用 JSON 文件实例化一个 `Wav2Vec2CTCTokenizer` 类的对象。

In [18]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

如果想在本notebook中使用刚刚创建的tokenizer和微调模型，强烈建议将`tokenizer`上传到[🤗 Hub](https://huggingface.co/)。我们将上传文件的仓库命名为`"wav2vec2-base-timit-demo-colab"`：

In [19]:
repo_name = "wav2vec2-base-timit-demo-google-colab"

好的，现在我们将Tokenizer上传到[🤗 Hub](https://huggingface.co/)。


In [34]:
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/weege007/wav2vec2-base-timit-demo-google-colab/commit/f426c023a2324254f41fc4c781b9a37a62daa5fd', commit_message='Upload tokenizer', commit_description='', oid='f426c023a2324254f41fc4c781b9a37a62daa5fd', pr_url=None, repo_url=RepoUrl('https://huggingface.co/weege007/wav2vec2-base-timit-demo-google-colab', endpoint='https://huggingface.co', repo_type='model', repo_id='weege007/wav2vec2-base-timit-demo-google-colab'), pr_revision=None, pr_num=None)

### Create Wav2Vec2 Feature Extractor

语音是一种连续信号，为了让计算机处理，它首先必须被离散化，这通常被称为**采样**。采样率在此起着重要作用，因为它定义了每秒测量多少个语音信号数据点。因此，更高的采样率采样会更好地近似*真实*的语音信号，但每秒也需要更多的数据值。

预训练的检查点期望其输入数据或多或少地从与其训练数据相同的分布中采样。以两种不同速率采样的相同语音信号具有非常不同的分布，例如，采样率加倍会导致数据点长度增加一倍。因此，在微调ASR模型的预训练检查点之前，验证用于预训练模型的数据采样率与用于微调模型的数据集的采样率是否匹配至关重要。

Wav2Vec2是在[LibriSpeech](https://huggingface.co/datasets/librispeech_asr)和LibriVox的音频数据上预训练的，两者都以16kHz采样。我们的微调数据集[Timit](https://www.google.com/search?q=https://huggingface.co/datasets/timit_asr)也很幸运地以16kHz采样。如果微调数据集的采样率低于或高于16kHz，我们首先必须对语音信号进行上采样或下采样，以匹配用于预训练数据集的采样率。

Wav2Vec2 特征提取器对象在实例化时需要以下参数：

* `feature_size`：语音模型以特征向量序列作为输入。虽然此序列的长度显然会有所不同，但特征大小不应改变。对于 Wav2Vec2，特征大小为 1，因为该模型是在原始语音信号上训练的。
* `sampling_rate`：模型训练时使用的采样率。
* `padding_value`：对于批量推理，较短的输入需要用特定值进行填充。
* `do_normalize`：输入是否应该进行*零均值单位方差*归一化。通常，归一化输入后，语音模型的性能会更好。
* `return_attention_mask`：模型在批量推理时是否应使用 `attention_mask`。通常，模型应**始终**使用 `attention_mask` 来掩盖填充的标记。然而，由于 `Wav2Vec2` “base”检查点的一个非常特殊的设计选择，在使用不带 `attention_mask` 的情况下能获得更好的结果。这不建议用于其他语音模型。有关更多信息，可以查看[此](https://github.com/pytorch/fairseq/issues/3227)问题。**重要提示**：如果想使用此 notebook 微调 [large-lv60](https://huggingface.co/facebook/wav2vec2-large-lv60)，则应将此参数设置为 `True`。

In [20]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [21]:
feature_extractor

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

很棒，Wav2Vec2 的特征提取管道由此完全定义！

为了尽可能方便用户使用 Wav2Vec2，特征提取器和分词器被**封装**到单个 `Wav2Vec2Processor` 类中，这样用户只需一个 `model` 和一个 `processor` 对象即可。

In [22]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [23]:
processor

Wav2Vec2Processor:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

- tokenizer: Wav2Vec2CTCTokenizer(name_or_path='', vocab_size=30, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	28: AddedToken("[UNK]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	29: AddedToken("[PAD]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	30: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	31: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False

好的，接下来我们可以准备数据集。


### 数据预处理

到目前为止，我们还没有查看语音信号的实际值，而只是查看了转录文本。除了`'text'`，我们的数据集中还包含另外两个列名`'file'`和`'audio'`。`'file'`表示音频文件的绝对路径。我们来看一下。

In [24]:
timit["train"][0]["file"]

'/root/.cache/huggingface/datasets/downloads/extracted/404950a46da14eac65eb4e2a8317b1372fb3971d980d91d5d5b221275b1fd7e0/data/TRAIN/DR4/MMDM0/SI681.WAV'

`Wav2Vec2` 要求输入是 16 kHz 的一维数组格式。这意味着必须加载并重新采样音频文件。

幸运的是，在调用 `audio` 列时，`datasets` 会自动完成此操作。我们来试一下。

In [25]:
timit["train"][0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/404950a46da14eac65eb4e2a8317b1372fb3971d980d91d5d5b221275b1fd7e0/data/TRAIN/DR4/MMDM0/SI681.WAV',
 'array': array([-2.1362305e-04,  6.1035156e-05,  3.0517578e-05, ...,
        -3.0517578e-05, -9.1552734e-05, -6.1035156e-05], dtype=float32),
 'sampling_rate': 16000}

我们可以看到音频文件已经自动加载了。这要归功于 `datasets == 4.13.3` 中引入的新 [`"Audio"` 特性](https://www.google.com/search?q=%5Bhttps://huggingface.co/docs/datasets/package_reference/main_classes.html%3Fhighlight%3Daudio%23datasets.Audio%5D\(https://huggingface.co/docs/datasets/package_reference/main_classes.html%3Fhighlight%3Daudio%23datasets.Audio\))，它在调用时会即时加载和重新采样音频文件。

采样率被设置为 16kHz，这正是 `Wav2Vec2` 所期望的输入。

很好，我们来听几个音频文件，以便更好地了解数据集并验证音频是否已正确加载。

**请注意**：*您可以多次点击以下单元格以收听不同的语音样本。*

In [26]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(timit["train"]))

print(timit["train"][rand_int]["text"])
print(len(timit["train"][rand_int]["audio"]["array"])/16000)
ipd.Audio(data=np.asarray(timit["train"][rand_int]["audio"]["array"]), autoplay=True, rate=16000)

move the garbage nearer to the large window 
2.72


好的，这是您内容的中文翻译：

可以听到，说话者的声音会随着语速、口音等的变化而改变。不过，总的来说，录音听起来相对清晰，这与朗读语音语料库的预期一致。

我们来做最后一次检查，通过打印语音输入的形状、其转录文本以及相应的采样率，以确保数据已正确准备。

**注意**：*您可以多次点击以下单元格以验证多个样本。*

In [27]:
rand_int = random.randint(0, len(timit["train"]))

print("Target text:", timit["train"][rand_int]["text"])
print("Input array shape:", np.asarray(timit["train"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", timit["train"][rand_int]["audio"]["sampling_rate"])

Target text: fortyseven states assign or provide vehicles for employees on state business 
Input array shape: (82944,)
Sampling rate: 16000


很好！一切看起来都很正常——数据是**一维数组**，**采样率始终对应 16kHz**，并且**目标文本也已标准化**。

最后，我们可以将数据集处理成模型训练所需的格式。我们将使用 `map(...)` 函数。

首先，我们通过简单地调用 `batch["audio"]` 来加载和重采样音频数据。
其次，我们从加载的音频文件中提取 `input_values`。在我们的例子中，`Wav2Vec2Processor` 仅对数据进行归一化。然而，对于其他语音模型，此步骤可能包括更复杂的特征提取，例如 [Log-Mel 特征提取](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum)。
第三，我们将转录文本编码为标签 ID。

**注意**：这个映射函数很好地展示了 `Wav2Vec2Processor` 类应该如何使用。在“正常”情况下，调用 `processor(...)` 会重定向到 `Wav2Vec2FeatureExtractor` 的调用方法。然而，当将处理器封装到 `as_target_processor` 上下文时，相同的方法会重定向到 `Wav2Vec2CTCTokenizer` 的调用方法。
欲了解更多信息，请查阅[文档](https://huggingface.co/docs/transformers/main/model_doc/wav2vec2#transformers.Wav2Vec2Processor.__call__)。

In [28]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

Let's apply the data preparation function to all examples.

In [29]:
import numpy as np
# fuck numpy
np.object = object

timit = timit.map(prepare_dataset, remove_columns=timit.column_names["train"], num_proc=4)

/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:170: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:170: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:170: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

**请注意**：目前 `datasets` 库使用 [`torchaudio`](https://www.google.com/search?q=%5Bhttps://pytorch.org/audio/stable/index.html%5D\(https://pytorch.org/audio/stable/index.html\)) 和 [`librosa`](https://www.google.com/search?q=%5Bhttps://librosa.org/doc/latest/index.html%5D\(https://librosa.org/doc/latest/index.html\)) 进行音频加载和重采样。如果您希望实现自己的自定义数据加载/采样方式，可以随意使用 `"path"` 列而忽略 `"audio"` 列。

由于长时间输入序列需要大量内存，并且 `Wav2Vec2` 基于 `self-attention` 机制，对于长输入序列，内存需求与输入长度呈平方级增长（*参见* [这篇 Reddit 帖子](https://www.reddit.com/r/MachineLearning/comments/genjvb/d_why_is_the_maximum_input_sequence_length_of/)）。为了本次演示，我们从训练数据集中过滤掉所有超过 4 秒的序列。

In [30]:
max_input_length_in_sec = 4.0
timit["train"] = timit["train"].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

  0%|          | 0/5 [00:00<?, ?ba/s]

太棒了，现在我们准备好开始训练了！

## 训练与评估

数据已经处理完毕，我们准备开始设置训练流程。我们将利用 🤗 的 [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) 类，为此我们基本上需要完成以下工作：

* **定义一个数据收集器（Data Collator）**。与大多数自然语言处理模型不同，Wav2Vec2 的输入长度远大于输出长度。例如，一个输入长度为 50000 的样本，其输出长度不会超过 100。考虑到巨大的输入尺寸，动态填充训练批次效率更高，这意味着所有训练样本都应该只填充到其批次中最长的样本，而不是整个数据集中最长的样本。因此，微调 Wav2Vec2 需要一个特殊的填充数据收集器，我们将在下面定义。

* **评估指标**。在训练过程中，模型应该以词错误率（Word Error Rate, WER）进行评估。我们应该相应地定义一个 `compute_metrics` 函数。

* **加载预训练检查点**。我们需要加载一个预训练检查点并为其正确配置训练。

* **定义训练配置**。

在模型微调完成后，我们将对测试数据进行正确评估，并验证它是否确实学会了正确转录语音。

### 设置 Trainer

让我们从定义数据收集器开始。数据收集器的代码复制自[这个例子](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81)。

不深入过多细节，与常见的数据收集器不同，这个数据收集器对 `input_values` 和 `labels` 进行不同的处理，并因此对它们应用独立的填充函数（同样利用了 Wav2Vec2 的上下文管理器）。这是必要的，因为语音输入和输出是不同的模态，这意味着它们不应该由相同的填充函数处理。
与常见的数据收集器类似，它会用 `-100` 填充标签中的 token，这样在计算损失时就**不会**考虑这些 token。

In [31]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [32]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [33]:
data_collator

DataCollatorCTCWithPadding(processor=Wav2Vec2Processor:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

- tokenizer: Wav2Vec2CTCTokenizer(name_or_path='', vocab_size=30, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	28: AddedToken("[UNK]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	29: AddedToken("[PAD]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	30: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	31: AddedToken("</s>", rstrip=F

接下来，定义评估指标。如前所述，ASR 中的主要指标是词错误率 (WER)，因此我们也将在本笔记中使用它。

In [34]:
# https://discuss.huggingface.co/t/cant-import-load-metric-from-datasets/107524/4
from datasets import load_metric

#import evaluate
#load_wer = evaluate.load("wer")


In [35]:
wer_metric = load_metric("wer")
#wer_metric = load_wer

模型将返回一个logit向量序列：
$\mathbf{y}_1, \ldots, \mathbf{y}_m$，其中 $\mathbf{y}_1 = f_{\theta}(x_1, \ldots, x_n)[0]$ 且 $n >> m$。

一个logit向量 $\mathbf{y}_1$ 包含我们之前定义的词汇表中每个词的对数几率，因此 $\text{len}(\mathbf{y}_i) =$ `config.vocab_size`。我们对模型最可能的预测感兴趣，因此取logits的 `argmax(...)`。此外，我们通过将 `-100` 替换为 `pad_token_id` 并解码ID，同时确保连续的token在CTC风格${}^1$中**不**被分组为相同的token，将编码后的标签转换回原始字符串。

In [36]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

现在，我们可以加载预训练的 `Wav2Vec2` 检查点。必须使用分词器的 `pad_token_id` 来定义模型的 `pad_token_id`，或者在 `Wav2Vec2ForCTC` 的情况下，也定义 CTC 的 *空白符 token* ${}^2$。为了节省 GPU 内存，我们启用 PyTorch 的[梯度检查点](https://pytorch.org/docs/stable/checkpoint.html)功能，并将损失削减方式设置为“*mean*”。

In [37]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:309: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
model

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder)

In [40]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

Total parameters: 94,396,320
Trainable parameters: 94,396,320


Wav2Vec2 的第一个组成部分由一堆 CNN 层组成，这些层用于从原始语音信号中提取具有声学意义但上下文独立的特征。模型的这一部分在预训练期间已经得到了充分的训练，并且正如[论文](https://arxiv.org/abs/2006.11477)中所述，不再需要进行微调。
因此，我们可以将*特征提取*部分的所有参数的 `requires_grad` 设置为 `False`。

In [41]:
model.freeze_feature_encoder()

最后一步，我们定义所有与训练相关的参数。
对其中一些参数进行更多解释：
* `group_by_length` 通过将输入长度相似的训练样本分批处理，使训练更高效。这可以显著加快训练时间，因为它大大减少了通过模型传递的无用填充 token 的总数。
* `learning_rate` 和 `weight_decay` 经过启发式调整，直到微调变得稳定。请注意，这些参数强烈依赖于 Timit 数据集，并且可能对其他语音数据集来说不是最优的。

有关其他参数的更多解释，可以查看[文档](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments)。

在训练期间，每 400 个训练步骤会异步上传一个检查点到 Hugging Face Hub。这使得您即使在模型仍在训练时，也可以使用演示小部件进行尝试。

**注意**：如果不想将模型检查点上传到 Hub，只需将 `push_to_hub=False`。

In [49]:
from transformers import TrainingArguments
from transformers.trainer_utils import IntervalStrategy

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=64,
  #evaluation_strategy="steps",
  eval_strategy=IntervalStrategy.STEPS,
  num_train_epochs=30,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

In [50]:
training_args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=500,
eval_strategy=IntervalStrategy.STEPS,
eval_use_gather_object=False,

现在，所有实例都可以传递给 `Trainer`，我们准备好开始训练了！

In [51]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=timit["train"],
    eval_dataset=timit["test"],
    tokenizer=processor.feature_extractor,
)

/tmp/ipython-input-51-3784996474.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [52]:
trainer

---

${}^1$ 为了使模型能够独立于说话者的语速，在 CTC 中，连续相同的标记会被简单地归为一个单一标记。然而，在解码时，编码后的标签不应该被分组，因为它们不对应于模型的预测标记，这就是为什么必须传递 `group_tokens=False` 参数。如果我们不传递这个参数，像“hello”这样的词将被错误地编码并解码为“helo”。

${}^2$ 空白标记允许模型通过强制在两个“l”之间插入空白标记来预测一个词，例如“hello”。我们的模型对“hello”的 CTC 一致性预测将是 `[PAD] [PAD] "h" "e" "e" "l" "l" [PAD] "l" "o" "o" [PAD]`。

### Training

训练将花费 90 到 270 分钟，具体取决于分配给此 notebook 的 GPU。虽然训练后的模型在 *Timit* 的测试数据上取得了令人满意的结果，但它绝不是一个经过最佳微调的模型。本 notebook 的目的是演示如何对 Wav2Vec2 的 [base](https://huggingface.co/facebook/wav2vec2-base)、[large](https://huggingface.co/facebook/wav2vec2-large) 和 [large-lv60](https://huggingface.co/facebook/wav2vec2-large-lv60) 检查点在任何英语数据集上进行微调。

如果您想使用此 Google Colab 来微调您的模型，您应该确保您的训练不会因不活动而停止。一个简单的防止方法是将以下代码粘贴到此标签的控制台（*右键点击 -> 检查 -> 控制台标签并插入代码*）。

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);
```

根据分配给您 Google Colab 的 GPU，您可能会在这里看到一个“out-of-memory”错误。在这种情况下，最好的办法可能是将 `per_device_train_batch_size` 减少到 16 甚至更小，并最终使用 [`gradient_accumulation`](https://huggingface.co/transformers/master/main_classes/trainer.html#trainingarguments)。

In [53]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:170: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
500,0.218200,0.372721,0.382055
1000,0.135200,0.407869,0.381779
1500,0.099200,0.438048,0.357866
2000,0.070000,0.460096,0.346565
2500,0.051400,0.437691,0.340845
3000,0.036300,0.446382,0.336848
3500,0.029500,0.457242,0.331955


/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:170: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:170: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs

TrainOutput(global_step=3750, training_loss=0.08702826194763183, metrics={'train_runtime': 4015.0043, 'train_samples_per_second': 29.724, 'train_steps_per_second': 0.934, 'total_flos': 3.0988662117946327e+18, 'train_loss': 0.08702826194763183, 'epoch': 30.0})

最终的词错误率 (WER) 应该在 0.3 左右，这是一个合理的数值，因为目前最先进的音素错误率 (PER) 略低于 0.1（参见[排行榜](https://paperswithcode.com/sota/speech-recognition-on-timit)），而且 WER 通常比 PER 更差。

您现在可以将训练结果上传到 Hub，只需执行以下指令：

In [54]:
trainer.push_to_hub()

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1753355642.feddc1943175.56468.0:   0%|          | 0.00/8.32k [00:00<?, ?B/s]

events.out.tfevents.1753356539.feddc1943175.56468.1:   0%|          | 0.00/10.8k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/weege007/wav2vec2-base-timit-demo-google-colab/commit/c2dbb8bdf910a7aeb39b558cfb5d3c7f33c3535d', commit_message='End of training', commit_description='', oid='c2dbb8bdf910a7aeb39b558cfb5d3c7f33c3535d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/weege007/wav2vec2-base-timit-demo-google-colab', endpoint='https://huggingface.co', repo_type='model', repo_id='weege007/wav2vec2-base-timit-demo-google-colab'), pr_revision=None, pr_num=None)


现在，您可以与所有的朋友、家人、心爱的宠物分享这个模型了：他们都可以使用标识符“your-username/the-name-you-picked”来加载它，例如：

```python
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("weege007/wav2vec2-base-timit-demo-google-colab")
processor = Wav2Vec2Processor.from_pretrained("weege007/wav2vec2-base-timit-demo-google-colab")
```

### 评估

在最后一部分，我们将在一些验证数据上运行我们的模型，以了解其效果如何。

让我们加载 `processor` 和 `model`。

In [56]:
processor = Wav2Vec2Processor.from_pretrained("weege007/wav2vec2-base-timit-demo-google-colab")

preprocessor_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json:   0%|          | 0.00/331 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/30.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

In [57]:
model = Wav2Vec2ForCTC.from_pretrained("weege007/wav2vec2-base-timit-demo-google-colab").cuda()

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

现在，我们将利用 `map(...)` 函数来预测每个测试样本的转录，并将预测结果保存在数据集本身中。我们将结果字典命名为 `"results"`。

**注意**：由于存在[此问题](https://github.com/pytorch/fairseq/issues/3227)，我们特意使用 `batch_size=1` 来评估测试数据集。由于填充的输入不会产生与未填充输入完全相同的输出，因此完全不填充输入可以获得更好的 WER。

In [58]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)

  return batch

In [59]:
results = timit["test"].map(map_to_result, remove_columns=timit["test"].column_names)

0ex [00:00, ?ex/s]

In [60]:
results

Dataset({
    features: ['pred_str', 'text'],
    num_rows: 1680
})

现在让我们计算一下整体的 WER。

In [61]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test WER: 0.375


官方[排行榜](https://paperswithcode.com/sota/speech-recognition-on-timit)。

让我们来看看一些预测，看看模型都犯了哪些错误。

In [67]:
show_random_elements(results)

,pred_str,text
0,fvbqmpahspz pmvpxaggwpabpvyrwpgaeprys pmkamp,fvbqmpahspz pmvpxaggwpabpvyrwpgaeprys pmkamp
1,zas pr mpjvgphkdevgtvrpmk phaz pahpcagr ahpvpz mybepf hyb fpfyhp,zas pryfpjvgphdeagptvurpmk phaz pahpcagpryfhpvzymmybepf hyebpfyhsp
2,nr ah pfyepzwpnamamvhpdnpt jvg pjgvhmp,nr ah pfyepzwpnvmamv hpdnpt jvg pjgvhmp
3,fvbqmpahspz pmvpxaggwpabpvyrwpgaeprys pmkamp,fvbqmpahspz pmvpxaggwpabpvyrwpgaeprys pmkamp
4,mk phzarrptvwpndmpmk puvgzpvbpmk pkvvsp,mk phzarrptvwpndmpmk puvgzpvbpmk pkvvsp
5,mvpjdvgmk gpkyhpnt ghm e pk pvxxahyvbarrwpg af hpmk puarrhmgwpcvdgbarp,mvpjdgmk gpkyhpng hmye pk pvxxahyvbarrwpg afhpmk puarrphmg mpcvdgbarp
6,hk pkafpwvdgpfagsphdympybpeg ahwpuahkpuam gparrpw agp,hk pkafpwvdgpfagsphdympybpeg ahwpuahkpuam gparrpw agp
7,fgvnpjyo pjvgzhpybpmk ptvlpt jvg pwvdpevpvdmp,fgvnpjyo pjvgzhpybpmk ptvlpt jvg pwvdpevpvdmp
8,n mdg yahpag pxvboy byabmpjvgpap'dyxsprdbxkp,nyii gyahpag pxvbo by bmpjvgpap'dyxsprdbxkp
9,ukwpe rrpvgpuvggwpvo gphxyrypymdzhp,ukwpw rrpvgpuvggwpvo gphyrrwpym zhp


很明显，预测的转录文本在声学上与目标转录文本非常相似，但经常包含拼写或语法错误。然而，考虑到我们纯粹依赖 Wav2Vec2 而没有使用语言模型，这不应该令人感到非常惊讶。

好的，最后，为了更好地理解 CTC 的工作原理，值得深入研究模型的精确输出。让我们将第一个测试样本通过模型运行，获取预测的 ID，并将其转换为相应的标记。

In [ ]:
model.to("cuda")

with torch.no_grad():
  logits = model(torch.tensor(timit["test"][:1]["input_values"], device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

'[PAD] [PAD] [PAD] [PAD] [PAD] t t h h e | | | b [PAD] [PAD] [PAD] u u n n g g [PAD] [PAD] l l l l [PAD] o o o | | w w a a s s | | [PAD] [PAD] [PAD] p l l [PAD] e s s s s [PAD] n n t t [PAD] l l l y y | | [PAD] s s s i i t t t [PAD] u u u u u [PAD] [PAD] [PAD] [PAD] a a t t [PAD] e e d d | | [PAD] n n e e a a r | | t t h e e | | s s h h [PAD] [PAD] [PAD] [PAD] o o r r r r | [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

输出结果应该能更清楚地说明 CTC 在实践中是如何工作的。模型在某种程度上对语速是不变的，因为它已经学会了在需要分类的语音片段仍对应相同标记的情况下，要么简单地重复相同的标记。这使得 CTC 成为语音识别中一个非常强大的算法，因为语音文件的转录通常与它的长度非常无关。

我再次建议读者查看[这篇](https://distill.pub/2017/ctc)非常棒的博客文章，以便更好地理解 CTC。